# SQL Notebook for SQL Server 2022 data virtualization for parquet files on Azure Data Lake Store

This is a SQL notebook to show the data virtualization capabilities in SQL Server 2022 for parquet files on Azure Data Lake storage. This notebook is based on  Exercise 5.4 of the SQL Server 2022 workshop at [https://aka.ms/sql2022workshop](https:\aka.ms\sql2022workshop) and from a Americas GPS PSA experience.

# Step 0 - Validate Polybase is Enabled

Run the following code to validate and enable Polybase if needed. Just once.

In [ ]:
EXEC sp_configure 'polybase enabled', 1;

In [ ]:
RECONFIGURE;

In [ ]:
EXEC sp_configure 'allow polybase export', 1;

In [ ]:
RECONFIGURE;

# Step 1 - Master Key

Create a master key to protect the database scoped credential. Edit the following statement to put in a strong password

In [ ]:
USE [polytest]
GO
IF NOT EXISTS (SELECT * FROM sys.symmetric_keys WHERE name = '##MS_DatabaseMasterKey##')
	CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>';
GO

# Step 2 - Database Scoped Credential

Create a database scoped credential. Edit the following statement to put in the Shared Access Signature from the Azure Storage Account. Remember that is SAS key has an experition day and should have the minimun access level required.

In [ ]:
USE [polytest];
GO
-- Have to drop the data source first if it exists
IF EXISTS (SELECT * FROM sys.external_data_sources WHERE name = 'AzureStorage')
	DROP EXTERNAL DATA SOURCE AzureStorage;
IF EXISTS (SELECT * FROM sys.database_scoped_credentials WHERE name = 'AzureStorageCredential')
    DROP DATABASE SCOPED CREDENTIAL AzureStorageCredential;
GO
CREATE DATABASE SCOPED CREDENTIAL AzureStorageCredential
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = '<SAS Key starting with "sv=">';
GO

# Step 3 - External Data Source

Create an external data source. Edit the script to put your Azure Storage Account (Data Lake) end point. Use "abs" end point. No type is needed on this case.

In [ ]:
USE [polytest];
GO
IF EXISTS (SELECT * FROM sys.external_data_sources WHERE name = 'AzureStorage')
	DROP EXTERNAL DATA SOURCE AzureStorage;
GO
CREATE EXTERNAL DATA SOURCE AzureStorage
WITH
(
      LOCATION ='abs://<container>@<storage account name>.blob.core.windows.net',  
      CREDENTIAL = AzureStorageCredential 
	  --, TYPE = HADOOP
);
GO

# Step 4 - External file format - optional, only if you will create External Tables

Create an external format file for parquet to be used for external tables

In [ ]:
USE [polytest];
GO
IF EXISTS (SELECT * FROM sys.external_file_formats WHERE name = 'ParquetFileFormat')
	DROP EXTERNAL FILE FORMAT ParquetFileFormat;
CREATE EXTERNAL FILE FORMAT ParquetFileFormat WITH(FORMAT_TYPE = PARQUET);
GO

# Step 5 - Use OPENROWSET() to query parquet files

Execute the following T-SQL to query parquet files stored in ADLS to test that all the configuraion variables are working. On the BULK paramenter add the path to where the PARQUET files are located.

In [ ]:
USE [polytest];
GO
SELECT top 10 *
FROM OPENROWSET
	(BULK '<rute to where the parquet files are located ej "/wwi/factsale">'
	, FORMAT = 'PARQUET'
	, DATA_SOURCE = 'AzureStorage')
as [wwi_sales];
GO

# Step 6 - Create a View

Execute the following T-SQL to create a view based on a <span style="color: var(--vscode-foreground);">query to parquet file</span>

In [ ]:
USE [polytest];
GO
CREATE VIEW <view name to create> AS 
SELECT *
FROM OPENROWSET
	(BULK '<rute to where the parquet files are located ej "/wwi/factsale">'
	, FORMAT = 'PARQUET'
	, DATA_SOURCE = 'AzureStorage')
as [wwi_sales];
GO

# Step 7 - Create an external table if you need a sub set of columns from the entire parquet on a table

Execute the following T-SQL to create an external table based on a parquet file with a subset of columns

In [ ]:
USE [polytest];
GO
IF OBJECT_ID('wwi_sales_table', 'U') IS NOT NULL
	DROP EXTERNAL TABLE wwi_sales_table;
GO
CREATE EXTERNAL TABLE wwi_sales_table 
( 
	SaleKey VARCHAR(8), 
	CustomerKey VARCHAR(8),
	StockItemKey VARCHAR(8),
	DeliveryDateKey VARCHAR(12),
	Profit decimal(18,2)
)
WITH 
(
	LOCATION = '/wwi/factsale-parquet2'
    , FILE_FORMAT = ParquetFileFormat
    , DATA_SOURCE = AzureStorage
);
GO
SELECT top 10 * FROM wwi_sales_table;
GO

In [ ]:
CREATE CLUSTERED COLUMNSTORE INDEX sales_index
ON wwi_sales_table;

In [ ]:
CREATE STATISTICS MyStats ON wwi_sales_table (CustomerKey);

# Step 8 - Create statistics on an external table

Execute the following T-SQL to create statistics on an external table

In [ ]:
USE [polytest];
GO
CREATE STATISTICS wwi_ctb_stats ON wwi_customer_transactions_base (CustomerID) WITH FULLSCAN;
GO

# Step 9 - Explorer data virtualization metadata

Execute the following T-SQL statements to explore data virtualization metadata

In [ ]:
USE [polytest];
GO
SELECT * FROM sys.external_data_sources;
GO
SELECT * FROM sys.external_file_formats;
GO
SELECT * FROM sys.external_tables;
GO

# Step 11 - Explore parquet file metadata

Execute the following T-SQL statement to explore metadata about columns in the parquet file

In [ ]:
USE [polytest];
GO
EXEC sp_describe_first_result_set N'
SELECT *
FROM OPENROWSET
	(BULK '<rute to where the parquet files are located ej "/wwi/factsale">'
	, FORMAT = 'PARQUET'
	, DATA_SOURCE = 'AzureStorage')
as [wwi_sales;';
GO

# Step 12 - Explore file metadata

Execute the following T-SQL statement to explore metadata from the external table about the source parquet files

In [ ]:
USE [polytest];
GO
SELECT TOP 1 wwi_customer_transactions_file.filepath(), 
wwi_customer_transactions_file.filename()
FROM OPENROWSET
	(BULK '/wwi/'
	, FORMAT = 'PARQUET'
	, DATA_SOURCE = 'AzureStorage')
as [wwi_customer_transactions_file];
GO